In [16]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn

In [17]:
data, target = load_boston(return_X_y=True)

df = pd.DataFrame(data)

X_train, X_test, y_train, y_test = train_test_split(df,target,random_state=0)

In [18]:
def calculate_errors(y,ypred):
    mae = mean_absolute_error(y,ypred)
    mse = mean_squared_error(y, ypred)
    R2 = r2_score(y,ypred)
    y,ypred = np.array(y), np.array(ypred)
    mape = np.mean(np.abs((y - ypred) / y)) * 100
#     print("MAE:{0:.3f}, MSE:{1:.2f}, R2:{2:.2f}".format(mae, mse, R2))
    return mae,mse,R2,mape

In [19]:
a = 0.1 
l = 0.5

en = ElasticNet(alpha=a,l1_ratio=l)
en.fit(X_train,y_train)
y_pred = en.predict(X_test)


In [20]:
mae,mse,R2,mape = calculate_errors(y_test,y_pred)
errors = mae,mse,R2,mape

## Train model

In [31]:
def train_elastic_net(a,l,exp = None):
    with mlflow.start_run(experiment_id=exp): #start mlflow run
        en = ElasticNet(alpha=a,l1_ratio=l)
        en.fit(X_train,y_train)
        y_pred = en.predict(X_test)
        
        #calculate errors
        mae,mse,R2,mape = calculate_errors(y_test,y_pred)
        errors = mae,mse,R2,mape
        print("MAE:{0:.3f}, MSE:{1:.2f}, R2:{2:.2f}".format(mae, mse, R2))
        
        #log metris and parmeters
        mlflow.log_metrics({"MAE":mae,"MSE":mse, "R2":R2, "MAPE":mape})
        mlflow.log_params({"alpha":a,"l1_ratio":l})
        
        #register model
        mlflow.sklearn.log_model(en, "model")
        
        #save error plot
        plt.figure()
        plt.bar(['mae','mse','R2','mape'],errors,color=['blue','red','green','orange']);
        plt.title("Errors")
        plt.savefig("errors.png")
        plt.close()
        mlflow.log_artifact("errors.png")

## Predict with different alpha and l1 ratio

In [32]:
 for a in np.arange(0.1, 1, 0.25):
        for l in np.arange(0.1, 1, 0.25):
            train_elastic_net(a,l)

MAE:3.787, MSE:32.25, R2:0.61
MAE:3.776, MSE:32.21, R2:0.61
MAE:3.764, MSE:32.16, R2:0.61
MAE:3.749, MSE:32.11, R2:0.61
MAE:3.882, MSE:33.66, R2:0.59
MAE:3.866, MSE:33.58, R2:0.59
MAE:3.845, MSE:33.41, R2:0.59
MAE:3.810, MSE:33.10, R2:0.59
MAE:3.936, MSE:34.57, R2:0.58
MAE:3.929, MSE:34.60, R2:0.58
MAE:3.914, MSE:34.49, R2:0.58
MAE:3.887, MSE:34.24, R2:0.58
MAE:3.966, MSE:35.23, R2:0.57
MAE:3.968, MSE:35.38, R2:0.57
MAE:3.970, MSE:35.47, R2:0.57
MAE:3.968, MSE:35.53, R2:0.57


In [33]:
print("target mean:{}".format(target.mean()))
print("target std:{}".format(target.std()))

target mean:22.532806324110677
target std:9.188011545278203


## new experiment

In [39]:
exp = mlflow.create_experiment(name="normalized data")

In [27]:
from sklearn import preprocessing

In [40]:
data = preprocessing.normalize(data)


In [36]:
data.shape

(506, 13)

In [37]:
 for a in np.arange(0.1, 1, 0.25):
        for l in np.arange(0.1, 1, 0.25):
            train_elastic_net(a,l,exp)

MAE:3.787, MSE:32.25, R2:0.61
MAE:3.776, MSE:32.21, R2:0.61
MAE:3.764, MSE:32.16, R2:0.61
MAE:3.749, MSE:32.11, R2:0.61
MAE:3.882, MSE:33.66, R2:0.59
MAE:3.866, MSE:33.58, R2:0.59
MAE:3.845, MSE:33.41, R2:0.59
MAE:3.810, MSE:33.10, R2:0.59
MAE:3.936, MSE:34.57, R2:0.58
MAE:3.929, MSE:34.60, R2:0.58
MAE:3.914, MSE:34.49, R2:0.58
MAE:3.887, MSE:34.24, R2:0.58
MAE:3.966, MSE:35.23, R2:0.57
MAE:3.968, MSE:35.38, R2:0.57
MAE:3.970, MSE:35.47, R2:0.57
MAE:3.968, MSE:35.53, R2:0.57
